In [47]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import uuid
from datetime import datetime
from sqlalchemy import create_engine, update, Table, Column, Integer, String, MetaData, inspect,text
import numpy as np
import json
import pyodbc

# Database configuration
db_config = {
    'dbname': 'import_final',
    'user': 'postgres',
    'password': '123',
    'host': 'localhost',
    'port': '5432'
}

postgres_db_config_mo = {
    'dbname': '67C_Monitoring',
    'user': 'postgres',
    'password': 'MyTTT%401234',
    'host': '203.154.82.165',
    'port': '5432'
}

postgres_db_config_med = {
    'dbname': 'meb',
    'user': 'postgres',
    'password': 'MyTTT%401234',
    'host': '203.154.82.165',
    'port': '5432'
}

sql_server_config_server = {
    'driver': '{SQL Server}',
    'server': '203.154.82.165',
    'database': 'response',
    'user': 'sa',
    'password': 'MyTTT@1234'
}


def setup_postgres_engine(db_config):
    try:
        db_url = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}"
        engine = create_engine(db_url)
        print("PostgreSQL connection successful")
        return engine
    except Exception as e:
        print(f"Error connecting to PostgreSQL: {e}")
        return None


def setup_sql_server_connection(db_config):
    try:
        conn_str = (
            f"DRIVER={db_config['driver']};"
            f"SERVER={db_config['server']};"
            f"DATABASE={db_config['database']};"
            f"UID={db_config['user']};"
            f"PWD={db_config['password']};"
        )
        conn = pyodbc.connect(conn_str)
        print("SQL Server connection successful")
        return conn
    except pyodbc.Error as e:
        print(f"Error connecting to SQL Server: {e}")
        return None
        
postgres_mybase = setup_postgres_engine(db_config)
postgres_67C_Monitoring = setup_postgres_engine(postgres_db_config_mo)
postgres_meb = setup_postgres_engine(postgres_db_config_med)
sql_server_resp = setup_sql_server_connection(sql_server_config_server)

PostgreSQL connection successful
PostgreSQL connection successful
PostgreSQL connection successful
SQL Server connection successful


-------

In [ ]:
control_query = "SELECT * FROM import_control_2024"
import_control_df = pd.read_sql(control_query, postgres_mybase) 

control_query = """SELECT * FROM response_import_2024_01_02"""
response_import_df = pd.read_sql(control_query, postgres_mybase) 

------

Send : meb_send 

In [ ]:
med_query = """SELECT "fileName", "messageId" FROM meb_sent"""
med_df = pd.read_sql(med_query, postgres_meb)

In [ ]:
# # Mapping fileName
ms_filename_to_id = med_df.set_index('fileName')['messageId'].to_dict()
import_control_df['ic_ms_messageid'] = import_control_df['ic_filename'].str.replace(r'\.sgn$', '', regex=True).map(ms_filename_to_id)
import_control_df['ic_filename'] = import_control_df['ic_filename'].str.replace(r'\.sgn$', '', regex=True) + '.sgn'

-----

Response : response, meb_inbox

In [ ]:
query_sql_resp = """SELECT "FileName", "InboxId" FROM response;"""
df_sql_resp = pd.read_sql(query_sql_resp, sql_server_resp)
df_sql_resp = df_sql_resp.astype(str)

query_meb_inbox = """SELECT "id", "reftomessageid", "messagetimestamp" FROM meb_inbox;"""
df_meb_inbox = pd.read_sql(query_meb_inbox, postgres_meb)
df_meb_inbox = df_meb_inbox.astype(str)

In [ ]:
# Mapping InboxId
map_inbox_id = df_sql_resp.set_index('FileName')['InboxId'].to_dict()
response_import_df['res_inbox_id'] = response_import_df['res_filename'].map(map_inbox_id)

# Mapping reftomessageid
map_message_id = df_meb_inbox.set_index('id')['reftomessageid'].to_dict()
response_import_df['res_message_id'] = response_import_df['res_inbox_id'].map(map_message_id)

In [ ]:
#หา FK ของ ic_id (res_ref_id)
df_map = import_control_df.set_index('ic_reference_number')['ic_id'].to_dict()
response_import_df['res_ref_id'] = response_import_df['res_ref_no'].map(df_map)

In [ ]:
response_import_df['res_data'] = response_import_df['res_data'].apply(json.dumps)
response_import_df.to_sql('response_import', postgres_mybase, if_exists='append', index=False, chunksize=1000)


-----

Declaration No. (PMTA,PMTR) : update ic_rs_id and ic_declaration_no 

In [ ]:
query = """SELECT rs_id, rs_type, rs_message, rs_status_id, rs_status_priority FROM response_status WHERE rs_declaration_type = 'Import';"""
df_response_status = pd.read_sql(query, postgres_67C_Monitoring)

In [ ]:
df_response_PAT = response_import_df[response_import_df['res_message_type'].isin(['PMTA', 'PMTR'])]
df_response_status_PMT = df_response_status[df_response_status['rs_type'].isin(['PMTA', 'PMTR'])] 
df_response_status_PMT = df_response_status_PMT.loc[df_response_status_PMT.groupby(['rs_type', 'rs_message'])['rs_status_priority'].idxmax()]

In [ ]:
merged_status = pd.merge(df_response_PAT, df_response_status_PMT, 
                        left_on=['res_message_type', 'res_message'], 
                        right_on=['rs_type', 'rs_message'],
                        how='left') 

ic_rs_id_map = merged_status.groupby('res_ref_no')['rs_id'].first()
ic_declaration_no_map = merged_status.groupby('res_ref_no')['res_declaration_number'].first()

import_control_df['ic_rs_id'] = import_control_df['ic_reference_number'].map(ic_rs_id_map)
import_control_df['ic_declaration_no'] = import_control_df['ic_reference_number'].map(ic_declaration_no_map)

--------

Cancel 

In [ ]:
res_query = """ SELECT * FROM cancel WHERE cc_declaration_type = 'IMP'
                AND cc_created_date BETWEEN '2024-01-01 00:00:00+00' AND '2024-02-29 23:59:59+00';"""
df_cancel_import = pd.read_sql(res_query, postgres_mybase) 

In [ ]:
# Mapping fileName
ms_filename_to_id = med_df.set_index('fileName')['messageId'].to_dict()
df_cancel_import['cc_messageid'] = df_cancel_import['cc_filename'].str.replace(r'\.sgn$', '', regex=True).map(ms_filename_to_id)
df_cancel_import['cc_filename'] = df_cancel_import['cc_filename'].str.replace(r'\.sgn$', '', regex=True) + '.sgn'

-----

Responce (XDCA,XCDR)

In [ ]:
res_query = """SELECT *,  res_data->>'DocumentNumber' AS DocumentNumber 
            FROM response_import 
            WHERE res_message_type = 'XCDA' OR res_message_type = 'XCDR';""" 

df_response_XCD = pd.read_sql(res_query, postgres_mybase) 

In [ ]:
# Mapping InboxId
map_inbox_id = df_sql_resp.set_index('FileName')['InboxId'].to_dict()
df_response_XCD['res_inbox_id'] = df_response_XCD['res_filename'].map(map_inbox_id)

# Mapping reftomessageid
map_message_id = df_meb_inbox.set_index('id')['reftomessageid'].to_dict()
df_response_XCD['res_message_id'] = df_response_XCD['res_inbox_id'].map(map_message_id)

----

Update : ic_is_cancel and ic_cancel_reason

In [ ]:
# map df_cancel_import['cc_messageid'] กับ df_response_XCD['res_message_id']
mapped_df = df_cancel_import[df_cancel_import['cc_messageid'].isin(df_response_XCD['res_message_id'])]

# เอา mapped_df ที่ import_control_df['ic_declaration_no'] ไป map df_cancel_import['cc_document_number']
for index, row in mapped_df.iterrows():
    ic_declaration_no = row['cc_document_number']

    # หาแถวที่ตรงกับ ic_declaration_no ใน import_control_df
    import_control_df.loc[import_control_df['ic_declaration_no'] == ic_declaration_no, 'ic_is_cancel'] = 't'
    import_control_df.loc[import_control_df['ic_declaration_no'] == ic_declaration_no, 'ic_cancel_reason'] = row['cc_cancel_reason']

---------

Declaration No. (IDCA,IDCA) : update ic_rs_id and ic_declaration_no 

In [ ]:
df_response_IDA = response_import_df[response_import_df['res_message_type'].isin(['IDCA', 'IDCR'])]
df_response_status_IDC = df_response_status[df_response_status['rs_type'].isin(['IDCA', 'IDCR'])] 
df_response_status_IDC = df_response_status_IDC.loc[df_response_status_IDC.groupby(['rs_type', 'rs_message'])['rs_status_priority'].idxmax()]

In [ ]:
merged_status = pd.merge(df_response_IDA, df_response_status_IDC, 
                        left_on=['res_message_type', 'res_message'], 
                        right_on=['rs_type', 'rs_message'],
                        how='left') 

ic_rs_id_map = merged_status.groupby('res_ref_no')['rs_id'].first()
ic_declaration_no_map = merged_status.groupby('res_ref_no')['res_declaration_number'].first()

import_control_df['ic_rs_id'] = import_control_df['ic_reference_number'].map(ic_rs_id_map)
import_control_df['ic_declaration_no'] = import_control_df['ic_reference_number'].map(ic_declaration_no_map)

----------

Update : ic_is_use

In [ ]:
# control_query = "SELECT * FROM response_import"
# response_import_df = pd.read_sql(control_query, postgres_mybase) 

In [ ]:
# Step 1: Convert 'ic_created_date' to datetime and create 'ic_is_use' based on 'ic_ms_messageid' being not null
import_control_df['ic_created_date'] = pd.to_datetime(import_control_df['ic_created_date'], errors='coerce')
import_control_df['ic_is_use'] = import_control_df['ic_ms_messageid'].notnull()

# Step 2: Handle rows with null 'ic_ms_messageid'
empty_ic_ms_messageid = import_control_df[import_control_df['ic_ms_messageid'].isnull()]
for ic_reference_number, group in empty_ic_ms_messageid.groupby('ic_reference_number'):
    if len(group) == 1:
        import_control_df.loc[group.index, 'ic_is_use'] = True
    else:
        most_recent_index = group['ic_created_date'].idxmax()
        import_control_df.loc[group.index, 'ic_is_use'] = False
        import_control_df.loc[most_recent_index, 'ic_is_use'] = True

# Step 3: Convert boolean 'ic_is_use' to string 't' or 'f'
import_control_df['ic_is_use'] = import_control_df['ic_is_use'].replace({True: 't', False: 'f'})


In [ ]:
# # Step 4: Handle duplicates and mismatches
duplicates = import_control_df[import_control_df.duplicated('ic_reference_number', keep=False)]
for ic_reference_number, group in duplicates.groupby('ic_reference_number'):
    if group['ic_ms_messageid'].isnull().any() and group['ic_ms_messageid'].notnull().any():
        import_control_df.loc[group[group['ic_ms_messageid'].isnull()].index, 'ic_is_use'] = 'f'

In [ ]:
# Additional Condition: Update 'ic_is_use' based on 'res_message_id' in 'response_import_df'
response_idca = response_import_df[response_import_df['res_message_type'] == 'IDCA']
response_message_ids_idca = set(response_idca['res_message_id'].values)

duplicate_groups = duplicates.groupby('ic_reference_number')
for ic_reference_number, group in duplicate_groups:
    if len(group) > 1:  # Only consider duplicates
        mask = group['ic_ms_messageid'].notnull() & ~group['ic_ms_messageid'].isin(response_message_ids_idca)
        import_control_df.loc[group[mask].index, 'ic_is_use'] = 'f'

In [ ]:
# import_control_df.to_sql('import_control', postgres_mybase, if_exists='append', index=False, chunksize=1000)


-----

Check : ic_is_use

In [ ]:
# กรณที่ ic_reference_number ไม่ซ้ำ 
import_control_df[import_control_df['ic_reference_number'] == 'DANU000105508']

In [ ]:
# กรณีที่ ic_reference_number ซ้ำ มี ic_ms_messageid
import_control_df[import_control_df['ic_reference_number'] == 'DJNU000016762']

In [ ]:
# กรณีที่ ic_reference_number ซ้ำ ไม่มี ic_ms_messageid
import_control_df[import_control_df['ic_reference_number'] == 'DMCO000025963']

In [ ]:
# กรณีที่มี ic_reference_number ซ้ำและมี ic_ms_messageid และไม่มี ic_ms_messageid
import_control_df[import_control_df['ic_reference_number'] == 'DAMK000019896']

In [ ]:
# ic_reference_number ซ้ำ ic_is_use ไม่มี t เลยสักตัว ไม่มี ic_ms_messageid เลยสักตัว
duplicate_ref_numbers = import_control_df[import_control_df.duplicated('ic_reference_number', keep=False)]
result = duplicate_ref_numbers.groupby('ic_reference_number').filter(
    lambda x: ~x['ic_is_use'].eq('t').any()
)

unique_refs = result['ic_reference_number'].unique()
unique_refs

In [ ]:
# ic_reference_number ซ้ำ ic_ms_messageid จับได้ทุกตัว ic_is_use = 't ทุกตัว
filtered_df = import_control_df[import_control_df['ic_is_use'] == 't']

duplicate_entries = filtered_df[filtered_df['ic_reference_number'].duplicated(keep=False)]
duplicate_entries


---